In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install scipy transformers
!pip install torch
!pip install pandas
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# import csv

# # Input and output file paths
# input_file = 'preprocessed_final_dataset.csv'
# output_file = 'output.csv'

# def enclose_in_quotes(field):
#     """Enclose a field in double quotes if it is not already enclosed."""
#     # Add quotes if not already enclosed
#     if not field.startswith('"') and not field.endswith('"'):
#         return f'"{field}"'
#     return field



# def process_file(file_path, output_file_path):
#     try:
#         with open(file_path, 'r') as file, open(output_file_path, 'w') as output_file:
#             for line in file:
#                 line = line.strip()  # Remove leading/trailing whitespace
#                 if line:  # Ensure the line is not empty
#                     # Split the line into two parts: before and after the last comma
#                     last_comma_index = line.rfind(',')
#                     if last_comma_index != -1:
#                         before_last_comma = line[:last_comma_index]
#                         after_last_comma = line[last_comma_index + 1:]

#                         before_last_comma = before_last_comma.replace('"', "'")

#                         if after_last_comma[0] != '"':
#                           output_file.write(f'"{before_last_comma}","{after_last_comma}"\n')
#                         else:
#                           output_file.write(f'"{before_last_comma}",{after_last_comma}\n')
#                     else:
#                         output_file.write(f'No comma found in line: {line}\n')
#     except FileNotFoundError:
#         print(f"Error: The file at {file_path} does not exist.")
#     except Exception as e:
#         print(f"An error occurred: {e}")

# # Example usage
# # Replace 'your_file.txt' and 'output_file.txt' with the paths to the files you want to process and write to
# process_file('final_dataset.csv', 'output.csv')






# print(f"Processing complete. Check the output in {output_file}.")


Processing complete. Check the output in output.csv.


In [ ]:
# import pandas as pd

# csv_file = "output.csv"
# expected_columns = 2

# # Read the file line by line
# with open(csv_file, 'r') as file:
#     for i, line in enumerate(file, start=1):
#         fields = line.strip().split(',')  # Split by commas
#         if len(fields) != expected_columns:
#             print(f"Line {i} has {len(fields)} fields: {line.strip()}")


Streaming output truncated to the last 5000 lines.
Line 11448 has 5 fields: "'''Where we stand today, Hillary Clinton winning Texas is very, very real,'' Anchia says. ''We have a candidate who has gone out of his way to alienate Hispanic voters.'","left"
Line 11449 has 5 fields: "'The efforts of Donald Trump and the Republicans, they're really delivering that vote in large percentages to the Democratic Party.'' The numbers from the first day of early voting in Texas were astonishing. Four years ago in Houston, 47,000 people voted on the first day.'","left"
Line 11450 has 5 fields: "'Peter Overby Campaign gear for sale before a campaign rally for Bernie Sanders in Oklahoma City, Oklahoma in February.J Pat Carter/Getty Imageshide caption Campaign gear for sale before a campaign rally for Bernie Sanders in Oklahoma City, Oklahoma in February. With Bernie Sanders lopping hundreds of staffers from his campaign this week, it's easy to forget he has outraised and outspent Hillary Clinton ever

In [ ]:
# import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, ClassLabel
import pandas as pd

csv_file = "preprocessed_final_dataset.csv"
df = pd.read_csv(csv_file)

print(df.columns)

label_mapping = {'left': 0, 'center': 1, 'right': 2}
df['label'] = df['Leaning'].map(label_mapping)

df = df[['Text', 'label']]

dataset = Dataset.from_pandas(df.rename(columns={'Text': 'text', 'label': 'label'}))
dataset = dataset.cast_column('label', ClassLabel(names=['left', 'center', 'right']))

Index(['Text', 'Leaning'], dtype='object')


Casting the dataset:   0%|          | 0/22069 [00:00<?, ? examples/s]

In [ ]:
print(dataset[4])
print(dataset[-4])

{'text': 'netanyahu that “he must pay more attention to the innocent lives being lost as a consequence of the actions taken, biden said', 'label': 1}
{'text': 'trump felt it was important to make the change now as he prepares for possible high stakes talks with north korean leader kim jong un as well as upcoming trade negotiations three white house officials said', 'label': 2}


In [ ]:
#!conda create -n bert_env python=3.9
#!conda activate bert_env
#!pip install "numpy<2" scipy transformers datasets pandas
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00


In [ ]:
# dataset["train"]

In [ ]:
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

dataset = dataset.train_test_split(test_size=0.8, stratify_by_column="label")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    # print(f"examples: {examples}   examples[text]: {examples['text']}")
    # for element in examples['text']:
        # print(f"examples[text] element: {element}")
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_dataset = dataset["train"].map(
    tokenize_function,
    batched=True,
    load_from_cache_file=True,
    batch_size=16,
)

eval_dataset = dataset["test"].map(
    tokenize_function,
    batched=True,
    load_from_cache_file=True,
    batch_size=16,
)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)

    acc = accuracy_score(labels, predictions)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="macro", zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "macro_f1": f1,
    }

training_args = TrainingArguments(
    learning_rate=1e-5,
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to=[],
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

eval_results = trainer.evaluate()

print(f"Evaluation Results: {eval_results}")


Map:   0%|          | 0/4413 [00:00<?, ? examples/s]

Map:   0%|          | 0/17656 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-5c4f06f01363>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Macro F1
1,0.727000,0.713584,0.695797,0.657319,0.640164,0.641677
2,0.580400,0.574052,0.742744,0.732568,0.731640,0.731614


Evaluation Results: {'eval_loss': 0.5740520358085632, 'eval_accuracy': 0.7427435432714091, 'eval_precision': 0.7325683570402471, 'eval_recall': 0.7316400006141711, 'eval_macro_f1': 0.7316140428078487, 'eval_runtime': 131.3203, 'eval_samples_per_second': 134.45, 'eval_steps_per_second': 8.407, 'epoch': 2.0}


In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')